 # Extra: voorbeeld Named Entity Recognition met een kaart

We kunnen Named Entity Recognition voor verschillende doeleinden gebruiken. Hieronder tonen we een voorbeeld waarin wwe de gevonden plaatsnamen uitzetten op een kaart. Zo krijg je in één overzicht direct een beeld van de verschillende locaties waarover in de kranten geschreven is. 

Als eerste importeer je de benodigde packages 

In [3]:
import pandas as pd
import spacy
import pickle
import plotly.express as px
import matplotlib.pyplot as plt
import folium

En daarna laad je het model. 

In [4]:
fiets_nlp = pd.read_pickle("data/fiets_nlp.pkl")

In [5]:
fiets_nlp.head(2)

,Unnamed: 0,identifier,type,title,date,content,subcategory,category,Year,DL score,spatial,length,doc
21061,84265,http://resolver.kb.nl/resolve?urn=MMKB23:00140...,artikel,BUITENLAND. POLITIEK OVERZICHT.,1880/05/26 00:00:00,"’s Gravenhage, 25 Mei.’t Is te Berlijn een lie...",fiets,fiets,1880,71.212121,Landelijk,792,"(’s, Gravenhage, ,, 25, Mei.’t, Is, te, Berlij..."
21072,84295,http://resolver.kb.nl/resolve?urn=ddd:01011723...,artikel,Gemengde Berichten.,1869/09/20 00:00:00,"Naar men verneemt, bloeit thans in den Hortus ...",rijwiel,fiets,1869,77.823129,Landelijk,735,"(Naar, men, verneemt, ,, bloeit, thans, in, de..."


De volgende functie wordt gebruikt om entities uit de `doc` kolom te halen. 

In [6]:
def get_ner(doc, entity):
    return [ent.text for ent in doc.ents if ent.label_ == entity]

In dit geval zijn we op zoek naar steden en dorpen, dus we maken enkel een kolom aan met de `GPE` entities. 

In [7]:
fiets_nlp['GPE'] = fiets_nlp['doc'].apply(lambda x: get_ner(x, 'GPE'))

Vervolgens laden we een csv bestand in met informatie over de coördinaten van een groot deel van de steden en (grotere) dorpen in de wereld. We gebruiken hiervoor de gratis versie van https://simplemaps.com/data/world-cities/ 

In [8]:
coordinates_df = pd.read_csv('data/worldcities.csv')
coordinates_df.head(2)

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.687,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
1,Jakarta,Jakarta,-6.175,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077


Vervolgens maken we een Python dictionary aan, met de mapping van de steden/dorpen en de latitude en longitude. 

In [9]:
city_to_coords = {row['city']: (row['lat'], row['lng']) for idx, row in coordinates_df.iterrows()}

Met onderstaande functie worden per locatie de coördinaten opgehaald uit bovenstaande dictionary. 

In [10]:
def get_gpe_coords(gpe_list):
    return [city_to_coords[gpe] for gpe in gpe_list if gpe in city_to_coords]

Vervolgens wordt een nieuwe kolom aangemaakt, waarop we deze functie toepassen. 

In [11]:
fiets_nlp['GPE_coords'] = fiets_nlp['GPE'].apply(get_gpe_coords)

Hieronder tonen we een gedeelte van het dataframe, met per artikel de kolom met de GP entiteiten en de kolom met bijbehorende coördinaten. 

In [13]:
display(fiets_nlp[['identifier','GPE','GPE_coords']].head())

,identifier,GPE,GPE_coords
21061,http://resolver.kb.nl/resolve?urn=MMKB23:00140...,"[Berlijn, Rome, Rome, nagaat, Granlle, Berlijn...","[(43.226, -75.4909), (43.226, -75.4909), (48.8..."
21072,http://resolver.kb.nl/resolve?urn=ddd:01011723...,"[Zwolle, Zondag, Gorinchem, Californië]","[(52.5167, 6.1), (51.8306, 4.9742)]"
21076,http://resolver.kb.nl/resolve?urn=ddd:01026324...,"[Frankrijk, Parijs, Parijs, Commun, Parijs, Ro...",[]
21078,http://resolver.kb.nl/resolve?urn=ddd:01011722...,"[gemeente Zype, Edinburg, Amerika, Parijs, Bar...","[(26.3196, -98.1597)]"
21079,http://resolver.kb.nl/resolve?urn=ddd:01009766...,"[Pari, afgeloopon, Neder_nd, Do Gazelle van An...","[(29.9707, -94.0015)]"


Vervolgens kunnen we deze coördinaten plotten op een kaart. Dit doen we met het package `folium`. 

In [14]:
# Plot the coordinates on a map
map = folium.Map(location=[52.0, 5.0], zoom_start=7)

for coords_list in fiets_nlp['GPE_coords']:
    for coords in coords_list:
        folium.Marker(location=coords).add_to(map)
        
# Save the map to an HTML file
map.save('map.html')

# Display the map
map

Nu zie je alle locaties op de kaart die in de artilelen genoemd worden. 

**Let op:** Als plaatsnamen OCR-fouten bevatten of oudere spelling hebben, dan kunnen ze niet worden gematched met het coördinatenbestand en komen ze dus niet voor op de map. 

Nu hebben alle punten op de map eenzelfde kleur. Je kunt hier nog meer onderscheid in aanbrengen, door bijvoorbeeld verschillende categorieën een veschillende kleuren te geven. 

Hieronder een voorbeeld van de punten op de map gekleurd naar verspreidingsgebied (spatial).

In [24]:
# Define a color mapping for each unique krantnaam
color_mapping = {
    'Landelijk': 'blue',
    'Regionaal/lokaal': 'purple'    
}

# Plot the coordinates on a map
map = folium.Map(location=[52.0, 5.0], zoom_start=7)

for index, row in fiets_nlp.iterrows():
    spatial = row['spatial']
    coords_list = row['GPE_coords']
    color = color_mapping.get(spatial, 'black')  # Default to black if krantnaam not in mapping
    
    for coords in coords_list:
        folium.Marker(location=coords, icon=folium.Icon(color=color)).add_to(map)

# Save the map to an HTML file
map.save('map.html')

# Display the map
map


En een kaart waarop de punten gekeurd zijn naar jaar. 

In [25]:
# Define a color mapping for each unique krantnaam
color_mapping = {
    1869: 'purple', 1870: 'green', 1871: 'red', 1872: 'pink', 1873: 'orange', 1874: 'blue',
    1875: 'darkpurple', 1876: 'lightblue', 1877: 'darkgreen', 1878: 'cadetblue', 1879: 'darkgreen',
    1880: 'darkblue', 1881: 'darkred', 1882: 'lightred', 1883: 'lightgreen', 1884: 'beige'
}

# Plot the coordinates on a map
map = folium.Map(location=[52.0, 5.0], zoom_start=7)

for index, row in fiets_nlp.iterrows():
    year = row['Year']
    coords_list = row['GPE_coords']
    color = color_mapping.get(year, 'black')  # Default to black if krantnaam not in mapping
    
    for coords in coords_list:
        folium.Marker(location=coords, icon=folium.Icon(color=color)).add_to(map)

# Save the map to an HTML file
map.save('map.html')

# Display the map
map
